# Tutorial #1: Selecting From the Library of Presimulated Components

**Target Audience:** Front-end User

**Table of Contents:**
1. [Selecting a the template of the component.](#component)
2. [Meaning / notation of characteristics associated with component.](#reading-characteristics)
3. [**[Main Functionality!]** Finding geometrical parameters which best suits target characteristics.](#select-characteristics)
4. [Using best geometry in `Qiskit Metal`](#qiskit-metal)


# Section 1: Selecting a Component <a id="component"></a>

First, let's see which components are currently supported.

In [2]:
import metal_library as mlib

print(mlib.supported_components)

['TransmonCross', 'TransmonPocket']


Now we can place one of the supported components into `Reader`. This is a class meant to parse the library (located at `C:\\...\metal_library\library`)

In [3]:
reader = mlib.Reader(component_name='TransmonCross')
print(reader)

Interfacing with the library will be conducted through this object.

# Section 2: Understanding Structure of Characteristics <a id="reading-characteristics"></a>

Since we're conducting science, we must use exact language when talking about the characteristics we're pulling.

Example of **imprecise** language:
- 👎 "Get the coupling strength?"

Example of **precise** language:
- 👍 "Get the coupling strength between the readout cavity and qubit"
- 👍 "Get the coupling strength between qubit #1 and the bus (connecting qubit #1 and qubit #2)"

The following attributes / functions were created to help the front-end user follow precisely what each component characteristic means.



### Component Types
In simulations, we don't always simulate the entire chip at once. So there's a property `component_types` to view which combinations we've simulated.

In [4]:
reader.component_types

╒════════════════════════════╤════════════════════════════════════════════════╕
│ Supported Component Type   │ Blurb                                          │
╞════════════════════════════╪════════════════════════════════════════════════╡
│ QubitOnly                  │ Qubit only                                     │
├────────────────────────────┼────────────────────────────────────────────────┤
│ QubitCavity                │ Qubit capacitvely coupled to a readout cavity. │
├────────────────────────────┼────────────────────────────────────────────────┤
│ QubitDriveline             │ Qubit capacitvely coupled to a drive line.     │
╘════════════════════════════╧════════════════════════════════════════════════╛


['QubitOnly', 'QubitCavity', 'QubitDriveline']

### Characteristics of Component Type

Within each `component_type`, the types of observables / component characteristics you can extract change. Obviously if you have simulated only a qubit you can't extract cavity resonant frequency. To see which characteristics are supported, we can use `Reader.get_characteristic_info`.

Note: `component_type` must be an entry in `Reader.component_types`

In [5]:
reader.get_characteristic_info(component_type="QubitOnly");

╒═════════════════════════╤═══════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name         │ Description                           │ Units   │ Math Symbol   │
╞═════════════════════════╪═══════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz     │ Qubit 01 Transition Freq (Linear GHz) │ GHz     │ f_{q,01}      │
├─────────────────────────┼───────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz │ Qubit Anharmonicity (Linear MHz)      │ MHz     │ \alpha_{q}    │
╘═════════════════════════╧═══════════════════════════════════════╧═════════╧═══════════════╛


In [6]:
reader.get_characteristic_info(component_type="QubitCavity");

╒═════════════════════════╤════════════════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name         │ Description                                    │ Units   │ Math Symbol   │
╞═════════════════════════╪════════════════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz     │ Qubit 01 Transition Freq (Linear GHz)          │ GHz     │ f_{q,01}      │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz │ Qubit Anharmonicity (Linear MHz)               │ MHz     │ \alpha_{q}    │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Cavity_Frequency_GHz    │ Cavity Resonant Freq (Linear MHz)              │ GHz     │ _{c}          │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Coupling_Strength_MHz   │ Coupling between qubit and cavity (Linear MHz

In [7]:
reader.get_characteristic_info(component_type="QubitDriveline");

╒══════════════════════════╤═════════════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name          │ Description                                 │ Units   │ Math Symbol   │
╞══════════════════════════╪═════════════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz      │ Qubit 01 Transition Freq (Linear GHz)       │ GHz     │ f_{q,01}      │
├──────────────────────────┼─────────────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz  │ Qubit Anharmonicity (Linear MHz)            │ MHz     │ \alpha_{q}    │
├──────────────────────────┼─────────────────────────────────────────────┼─────────┼───────────────┤
│ Driveline_Decay_Rate_MHz │ Feedline limited T1 Decay Rate (linear MHz) │ MHz     │ \kappa        │
╘══════════════════════════╧═════════════════════════════════════════════╧═════════╧═══════════════╛


# Section 3: (MAIN FUNCTIONALITY) Finding a Geometry for Given Characteristics <a id="select-characteristics"></a>

Now that we understand which `component_name` and `component_type` gives us the parameters we desire. We can begin to select geometries which will yield promising results.

First, you must select a `component_type` and prepare the library for the selection class (`Selector`).

In [8]:
reader = mlib.Reader(component_name='TransmonCross')
reader.read_library(component_type='QubitOnly')

Note: if you failed to run the previous cell, you haven't selected a `component_type`, and `Selector` will throw an error like:
```
reader = mlib.Reader(component_name='TransmonCross')
selector = mlib.Selector(reader)

>> AttributeError: '`Reader` must have `Reader.library` created. Run `Reader.read_library` before initalizing `Selector`.'

```

Now that everything is loaded, we can initalize the `Selector` class.

In [9]:
#### REMINDER: don't skip the previous cell! 
selector = mlib.Selector(reader)
print(selector)

### Let's select a geometry which results in the closest qubit characteristics.

Call `Selector.find_closest`

Documentation:

```
Main functionality. Select the closest geometry for a set of characteristics.

Args:
    target_params (dict): A dictionary where the keys are the column names in `self.characteristic`,
                          and the values are the target values to compare against.
    num_top (int): The number of rows with the smallest Euclidean distances to return.
    metric (str, optional): Metric to determine closeness. Defaults to "Euclidian". 
                            Must choose from `self.__supported_metrics__`.
    display (boo, optional): Print out results? Defaults to True.

Returns:
    indexes_smallest (pd.Index): Indexes of the 'num_top' rows with the smallest distances to the target parameters.
    best_characteristics (list[dict]): Associated characteristics. Ranked closest to furthest, same order as `best_geometries`
    best_geometries (list[dict]): Geometries in the style of QComponent.options. Ranked closest to furthest.
```


In [19]:
# You can find the supported metrics here...
selector.__supported_metrics__

['Euclidian', 'Manhattan', 'Chebyshev']

In [18]:
indexes, chars, best_geoms = selector.find_closest(target_params={"Qubit_Frequency_GHz": 5,
                                                                  "Qubit_Anharmonicity_MHz": 200},
                                                   num_top=3,
                                                   metric="Euclidian",
                                                   display=True)

INFO:root:NOTE TO USER: the value 5 for Qubit_Frequency_GHz is outside the bounds of our library.
INFO:root:If you find a geometry which corresponds to these values, please consider contributing it! 😁🙏


TESTING TESTING
Here are the closest 3 geometries
Target parameters: {'Qubit_Frequency_GHz': 5, 'Qubit_Anharmonicity_MHz': 200}
Metric: Euclidian


Ranking (Closest to Furthest),Index,Characteristic from Library,Geometry from Library
1,625,"{'Qubit_Frequency_GHz': 3.926279169, 'Qubit_Anharmonicity_MHz': 199.216393, 'misc': '{'project_info': {'pinfo': design_name Qubit_Sabrina_design project_name Qubit_Sabrina_proj project_path D:/lfl/Documents/Ansoft/ setup_name TransmonSetup dtype: object, 'dissip': dielectrics_bulk None dielectric_surfaces [layer_3_datatype_0_plane] resistive_surfaces None seams None dtype: object, 'options': Series([], dtype: float64), 'junctions': j0 Cj_variable Cj_0 Lj_variable Lj_0 length 0.0001 line JJ_Lj_1_rect_jj_ rect JJ_rect_Lj_1_rect_jj, 'ports': Empty DataFrame Columns: [] Index: []}, 'results': {'0': {'Pm': j0 0 0.974385, 'Pm_cap': j0 0 0.018225, 'Sm': s_j0 0 -1, 'Om': 0 freq_GHz 4.113516, 'sols': U_H U_E 0 5.978363e-28 8.198084e-26, 'Qm_coupling': Empty DataFrame Columns: [] Index: [0], 'Ljs': j0 1.400000e-08 dtype: float64, 'Cjs': j0 2.000000e-15 dtype: float64, 'Qs': 0 inf dtype: float64, 'freqs_hfss_GHz': 0 4.113516 dtype: float64, 'hfss_variables': _Cj_0 0fF _Lj_0 14nH dtype: object, 'modes': range(0, 1), 'I_peak': 0 0 -2.410747e-09 dtype: float64 dtype: object, 'V_peak': 0 0 -8.723138e-07 dtype: float64 dtype: object, 'ansys_energies': {0: {'U_J_inds': {'j0': 4.0681903237364056e-26}, 'U_J_caps': {'j0': 7.609314160650732e-28}, 'U_H': 2.98918152440136e-28, 'U_E': 4.099042226222435e-26, 'U_tot_ind': 4.098082138980419e-26, 'U_tot_cap': 4.175135367828943e-26, 'U_norm': 4.175135367828943e-26, 'U_diff': 0.009313574650382895}}, 'mesh': Unnamed: 0 Num Tets Min edge length \ 0 layer_1_datatype_0_plane 2706 0.001787 1 layer_3_datatype_0_plane 59631 0.000467 2 vacuum_box 46845 0.000443 3 cross_transmon 2217 0.001281 4 readout_connector_arm_transmon 113 0.011225 Max edge length RMS edge length Min tet vol Max tet vol \ 0 0.347215 0.068460 2.316180e-10 2.789200e-05 1 0.604401 0.037494 3.770610e-12 6.586310e-03 2 1.388860 0.047152 2.066530e-12 4.793560e-02 3 0.042426 0.013548 5.522210e-11 3.000000e-07 4 0.055000 0.027009 7.688870e-09 1.833330e-07 Mean tet vol Std Devn (vol) 0 6.546870e-07 2.071090e-06 1 1.210220e-05 1.438690e-04 2 3.385300e-05 7.335300e-04 3 2.029770e-08 2.997660e-08 4 4.070800e-08 3.179760e-08 , 'convergence': Solved Elements Max Delta Freq. % Pass Number 1 1184 NaN 2 1529 89.905000 3 1962 17.052000 4 2525 4.913400 5 3265 2.708600 6 4207 1.583600 7 5405 1.051300 8 6940 0.957230 9 8963 0.863090 10 11528 0.596880 11 14814 0.440010 12 19110 0.403560 13 24635 0.306180 14 31746 0.250820 15 40864 0.241520 16 52065 0.187770 17 67118 0.147920 18 82095 0.105880 19 95759 0.080445 20 106476 0.051609, 'convergence_f_pass': None}}}'}","{'pos_x': 0, ' pos_y': '0.0um', ' orientation': 0, ' chip': 'main', ' layer': 1, ' connection_pads': {'readout': {'connector_type': 0, 'claw_length': '215um', 'ground_spacing': '7um', 'claw_width': '10um', 'claw_gap': '5.1um', 'claw_cpw_length': '40um', 'claw_cpw_width': '10um', 'connector_location': 180}}, ' cross_width': '30um', ' cross_length': '195um', ' cross_gap': '29um', ' hfss_inductance': '10nH', ' hfss_capacitance': 0, ' hfss_resistance': 0, ' hfss_mesh_kw_jj': 7e-06, ' q3d_inductance': '10nH', ' q3d_capacitance': 0, ' q3d_resistance': 0, ' q3d_mesh_kw_jj': 7e-06, ' gds_cell_name': 'my_other_junction', ' aedt_q3d_inductance': 1e-08, ' aedt_q3d_capacitance': 0, ' aedt_hfss_inductance': 1.4e-08, ' aedt_hfss_capacitance': 0}"
2,352,"{'Qubit_Frequency_GHz': 3.926279169, 'Qubit_Anharmonicity_MHz': 199.216393, 'misc': '{'project_info': {'pinfo': design_name Qubit_Sabrina_design project_name Qubit_Sabrina_proj project_path D:/lfl/Documents/Ansoft/ setup_name TransmonSetup dtype: object, 'dissip': dielectrics_bulk None dielectric_surfaces [layer_3_datatype_0_plane] resistive_surfaces None seams None dtype: object, 'options': Series([], dtype: float64), 'junctions': j0 Cj_variable Cj_0 Lj_variable Lj_0 length 0.0001 line JJ_Lj_1_rect_jj_ rect JJ_rect_Lj_1_rect_jj, 'ports': Empty DataFrame Columns: [] Index: []}, 

# Section 4: Using these results in `Qiskit Metal`<a id="qiskit-metal"></a>

Now that we have acquired the best geometry, implementation in Qiskit Metal is super easy!

#### First, make your usual imports.

In [ ]:
%load_ext autoreload
%autoreload 2

# Qiskit Metal imports
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict

design = designs.DesignPlanar()

gui = MetalGUI(design)

**Second, you should double check which component you ended up interfacing with.**

In [14]:
reader.component_name

'TransmonCross'

Ah yes, it was the `TransmonCross`!

**Third, import this and parse our the results from `Selector.find_closest`**.

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross

# Parsing the best geometries 
best_options        = best_geoms[0]
second_best_options = best_geoms[1]
third_best_options  = best_geoms[2]
# and so on.. so on...

TransmonCross(design, "transmon", options=best_options)

gui.rebuild()
gui.zoom_on_components(['transmon'])
gui.screenshot()

🎉🥳 Congrats on learning how to use the main functionality of `metal-library`.

If you have the time, we'd greatly appreciate any contributions to the components library! Not only does this help you design the ,If that sounds interesting 

# Section 5. Qubit & Cavity Selection

**NOTE: Still needs to be implemented**

In [ ]:
reader = mlib.Reader(component_name='TransmonCross')
reader.read_library(component_type='QubitCavity') # Note, we've change it from "QubitOnly" -> "QubitCavity"

selector = mlib.Selector(reader)

In [ ]:
indexes, chars, best_geoms = selector.find_closest(target_params={"Qubit_Frequency_GHz": 5,
                                                                  "Qubit_Anharmonicity_MHz": 200,
                                                                  "Cavity_Frequency_GHz": 7,
                                                                  "Coupling_Strength_MHz": 70,
                                                                  "Cavity_Wavelength": "half", # acceptable inputs ["half", "quarter"], refers to wavelength of resonator
                                                                  "Feedline_Coupling": 'capacitive' #acceptable inputs ["capacitive", "inductive"], refers to 
                                                                  } 
                                                   num_top=3,
                                                   metric="Euclidian",
                                                   display=True)

In [ ]:
%load_ext autoreload
%autoreload 2

# Qiskit Metal imports
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict

design = designs.DesignPlanar()

gui = MetalGUI(design)

In [ ]:
from metal_library.components import QubitCavity

# Parsing the best geometries 
best_options        = best_geoms[0]
second_best_options = best_geoms[1]
# and so on...

design = QubitCavity(design, options=best_options)


gui.rebuild()
gui.autoscale()
gui.screenshot()